In [3]:
import pandas as pd
d = pd.read_csv("data/local_topics.csv")
d["label"] = d["Onderwerp"]
d = d[["text", "label"]]
d.head(10)

,text,label
0,Voorzichtig haalde Clim Backes van het Jocus M...,2501 - Cultuur en media
1,11 blikken schilden met een geschilderde afbee...,2501 - Cultuur en media
2,Fotoclub Venlo bestaat 100 jaar dit jaar en vi...,2501 - Cultuur en media
3,De melding van het ongeval kwam rond 16.00 uur...,2500 - Ongevallen
4,De twee Blerickenaren worden op vrijdag 20 jan...,2499 - Jubilea en herdenkingen
5,Na vanavond is de gemeente Venlo dus drie prin...,2503 - Evenementen en uitgaan
6,&nbsp;De oud-speler van VVV ontving de waarder...,2499 - Jubilea en herdenkingen
7,"Roemer stond samen met Bart Storcken, Emil Sza...",2503 - Evenementen en uitgaan
8,De winst tegen de tegenstander uit Beek beteke...,2495 - Sport
9,Elke zaterdag om 11:00 uur live op de radio pr...,2503 - Evenementen en uitgaan


In [4]:
d["label"].value_counts()

2500 - Ongevallen                   28
2503 - Evenementen en uitgaan       26
2501 - Cultuur en media             23
2495 - Sport                        20
2502 - Vervoer en werkzaamheden     19
2494 - Criminaliteit en justitie    19
2497 - Natuur en milieu             15
2498 - Bedrijfsleven                15
2493 - Gezondheidszorg en corona    11
2499 - Jubilea en herdenkingen       8
Politiek                             4
2496 - Onderwijs                     3
Name: label, dtype: int64

In [5]:
id2label = {1: "Ongevallen", 2: "Evenementen en uitgaan", 3: "Cultuur en media", 4:"Sport", 5:"Vervoer en werkzaamheden", 6:"Criminaliteit en justitie", 7:"Natuur en milieu", 8:"Bedrijfsleven", 9:"Gezondheidszorg en corona", 10: "Jubilea en herdenkingen", 11: "Politiek", 12:"Onderwijs"}
label2id = {v:k for (k,v) in id2label.items()}

In [7]:
from datasets import Dataset
ds = Dataset.from_pandas(d).train_test_split(test_size=.3)
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 133
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 58
    })
})

In [8]:
from transformers import AutoTokenizer, AutoModel
checkpoint = "GroNLP/bert-base-dutch-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


2023-01-22 20:21:21.775578: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-22 20:21:22.883402: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-22 20:21:22.883478: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-22 20:21:22.883486: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [17]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)
tokenized = ds.map(preprocess_function, batched=True)
tokenized = tokenized.remove_columns(ds["train"].column_names)
tokenized

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 133
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 58
    })
})

In [18]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [23]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
model = AutoModelForSequenceClassification.from_pretrained(
   checkpoint, num_labels=12, id2label=id2label, label2id=label2id
)

loading configuration file config.json from cache at /home/nel/.cache/huggingface/hub/models--GroNLP--bert-base-dutch-cased/snapshots/484ff5cec2ad42b434537dadd901d9b8e2b64cd2/config.json
Model config BertConfig {
  "_name_or_path": "GroNLP/bert-base-dutch-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "1": "Ongevallen",
    "2": "Evenementen en uitgaan",
    "3": "Cultuur en media",
    "4": "Sport",
    "5": "Vervoer en werkzaamheden",
    "6": "Criminaliteit en justitie",
    "7": "Natuur en milieu",
    "8": "Bedrijfsleven",
    "9": "Gezondheidszorg en corona",
    "10": "Jubilea en herdenkingen",
    "11": "Politiek",
    "12": "Onderwijs"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "Bedrijfsleven": 8,
    "Criminaliteit en justitie": 6,

In [24]:
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [25]:
from transformers import Trainer
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized['train'],
    eval_dataset=tokenized['test'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [26]:
trainer.train()

***** Running training *****
  Num examples = 133
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 399
  Number of trainable parameters = 109146636


ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,token_type_ids,attention_mask.

In [24]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

NameError: name 'tokenized_datasets' is not defined

In [23]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)
print(preds)

[1 0 1 1 0 1 0 0 1 0 1 0 0 1 0 1 1 1 0 0 0 1 1 1 1 0 0 1 0 0 1 0 1 0 0 0 1
 0 1 1 1 1 1 1 0 0 0 1 1 0 0 1 1 1 0 1 0 0 0 0 1 0 1 1 1 0 1 1 1 0 0 1 1 0
 0 1 0 1 1 0 1 0 0 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 1 1 0 1 0 0 1 0
 0 0 0 1 1 1 1 1 0 1 1 1 0 1 1 0 0 1 1 0 0 1 0 0 1 0 0 1 0 0 0 1 1 0 0 1 1
 0 1 1 1 1 0 1 0 1 0 0 1 0 0 0 0 1 0 0 0 1 1 1 1 1 0 1 1 0 0 0 1 0 0 0 1 0
 1 1 1 0 0 0 1 1 1 1 1 0 0 1 0 1 1 0 0 1 0 1 1 0 1 1 0 0 0 0 1 1 0 1 1 1 0
 1 1 1 1 0 1 1 0 0 1 0 0 1 1 1 0 0 1 1 0 0 1 1 1 1 0 0 1 1 0 1 0 0 0 0 0 0
 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 1 1 1 1 0 1 1 0 0 1 0 0 1 1 1 1 1 0 1 1 1 1
 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 0 1 0 0 1 0 1 1 1 0 0 0 1 1
 1 1 0 1 0 0 1 0 1 0 0 1 1 0 0 0 0 0 1 1 1 1 0 0 0 1 1 1 0 1 0 0 0 1 1 0 1
 0 1 1 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0 1 0 0 0 1 1 1 1 1 1 1 1 1 0 0 1 1 1
 0 0 0 1 1 0 0 1 1 1 1 1 0 0 1 1 0 1 1 1 1 1 0 1 0 0 1 0 1 0 1 1 1 1 0 0 1
 0 1 0 0 1 1 1 1 0 1 0 1 0 1 1 1 1 0 0 0 1 0 1 1 1 0 1 0 1 1 0 1 0 0 0 1 1
 1 1 1 1 1 0 0 1 0 0 0 1 

In [31]:
import evaluate

metric = evaluate.load("glue", "sst2")
print(metric)
metric.compute(predictions=preds, references=predictions.label_ids)

EvaluationModule(name: "glue", module_type: "metric", features: {'predictions': Value(dtype='int64', id=None), 'references': Value(dtype='int64', id=None)}, usage: """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
Examples:

    >>> glue_metric = evaluate.load('glue', 'sst2')  # 'sst2' or any of ["mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=ref

{'accuracy': 0.9002293577981652}

In [25]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "sst2")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [26]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file config.json from cache at /home/nel/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_emb

In [27]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: idx, sentence. If idx, sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/nel/letshug/env/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 67349
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 25257
  Number of trainable parameters = 109483778


Epoch,Training Loss,Validation Loss,Accuracy
1,0.269500,0.444317,0.885321
2,0.179200,0.462873,0.895642
3,0.113200,0.440344,0.900229


Saving model checkpoint to test-trainer/checkpoint-500
Configuration saved in test-trainer/checkpoint-500/config.json
Model weights saved in test-trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-500/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-1000
Configuration saved in test-trainer/checkpoint-1000/config.json
Model weights saved in test-trainer/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-1500
Configuration saved in test-trainer/checkpoint-1500/config.json
Model weights saved in test-trainer/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-1500/tokenizer_config.json
Special tokens file saved

Configuration saved in test-trainer/checkpoint-11500/config.json
Model weights saved in test-trainer/checkpoint-11500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-11500/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-11500/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-12000
Configuration saved in test-trainer/checkpoint-12000/config.json
Model weights saved in test-trainer/checkpoint-12000/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-12000/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-12000/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-12500
Configuration saved in test-trainer/checkpoint-12500/config.json
Model weights saved in test-trainer/checkpoint-12500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-12500/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-12500/spec

Configuration saved in test-trainer/checkpoint-22500/config.json
Model weights saved in test-trainer/checkpoint-22500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-22500/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-22500/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-23000
Configuration saved in test-trainer/checkpoint-23000/config.json
Model weights saved in test-trainer/checkpoint-23000/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-23000/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-23000/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-23500
Configuration saved in test-trainer/checkpoint-23500/config.json
Model weights saved in test-trainer/checkpoint-23500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-23500/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-23500/spec

TrainOutput(global_step=25257, training_loss=0.20143393338960836, metrics={'train_runtime': 2681.2745, 'train_samples_per_second': 75.355, 'train_steps_per_second': 9.42, 'total_flos': 3088292734504560.0, 'train_loss': 0.20143393338960836, 'epoch': 3.0})

In [32]:
predictions = trainer.predict(tokenized_datasets["validation"])
preds = np.argmax(predictions.predictions, axis=-1)
metric.compute(predictions=preds, references=predictions.label_ids)

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: idx, sentence. If idx, sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 872
  Batch size = 8


{'accuracy': 0.9002293577981652}